### Preliminaries

In [1]:
import mlflow
from mlflow.tracking import MlflowClient

import os, sys

import torch
import torch.nn.functional as F

HOME = os.environ["HOME"]
CARDIAC_COMA_REPO = f"{HOME}/01_repos/CardiacCOMA/"
import os; os.chdir(CARDIAC_COMA_REPO)

from config.cli_args import overwrite_config_items
from config.load_config import load_yaml_config, to_dict

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image

import pandas as pd
import shlex
from subprocess import check_output

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

#import surgeon_pytorch
#from surgeon_pytorch import Inspect, get_layers

import numpy as np
import pandas as pd
from IPython import embed
sys.path.insert(0, '..')

# import model.Model3D
# from utils.helpers import get_coma_args, get_lightning_module, get_datamodule
from copy import deepcopy
from pprint import pprint

from typing import List
from tqdm import tqdm

import pandas as pd

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual

In [6]:
import os; os.chdir(f"{HOME}/01_repos/CardiacGWAS/")


In [2]:
import glob
import os
import numpy as np
import pandas as pd
from IPython import embed
import functools

from data.gwas_details import GWAS_dict
GWAS_PATH = "data/other_gwas/original_files"

In [7]:
gwas_harmonized_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}.tsv"
gwas_selected_snps_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}__selected_snps.tsv"
COMA_GWAS_SUMMARY = "results/gwas_loci_summary_across_runs.csv"
LOGP_PATH = "results/log10p_for_selected_snps_across_gwas.csv"

### GWAS file - reduction

In [4]:
for file, info in GWAS_dict.items():
    prefix = info["prefix"]
    phenotype = info["phenotype"].replace(" ", "_")
    o_filename = f"{prefix}__{phenotype}.tsv"
    file = os.path.join(GWAS_PATH, file)
    df = pd.read_csv(file, sep="\t").rename(info["columns"], axis=1)    
    df = df[["CHR", "BP", "SNP", "P"]]
    df.to_csv(o_filename, sep='\t', header=True, index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/other_gwas/original_files/Aung2019/AungN_31554410_LVMVR.txt'

### Filter GWAS files for specific SNPS

#### Find significant SNPs across COMA runs.

In [8]:
gwas_loci_summary_across_runs_df = pd.read_csv(COMA_GWAS_SUMMARY)
idx = gwas_loci_summary_across_runs_df.groupby(["region"])["P"].transform(min) == gwas_loci_summary_across_runs_df["P"]

best_association_per_region = gwas_loci_summary_across_runs_df[idx].sort_values("region")
best_snps = set(best_association_per_region.SNP)

In [9]:
from PIL import Image

In [10]:
best_association_per_region

,run,pheno,region,CHR,SNP,BP,AF,a_0,a_1,BETA,SE,T,P,locus_name
1012,e6490cf8389f4e7da52fef013562770f,1_e6490_z012,chr10_20,10.0,rs2797593,28655648.0,0.358680,A,C,0.049402,0.008336,5.9265,3.126079e-09,Unnamed
593,8b630b3a67f94ec2853645fe0314a424,1_8b630_z003,chr10_69,10.0,rs189569984,112544125.0,0.008998,C,T,-0.246570,0.041965,-5.8757,4.253045e-09,RBM20
815,b0ca240bf966455888005eee1e1fed99,1_b0ca2_z004,chr10_74,10.0,rs375034445,121424815.0,0.214880,A,AT,-0.056170,0.009752,-5.7600,8.489850e-09,Unnamed
744,a7d3200ce0fb43d586d53db5afff50d9,1_a7d32_z005,chr11_2,11.0,rs569550,1887068.0,0.386020,T,G,-0.057778,0.008179,-7.0643,1.648162e-12,LSP1*
889,cc438c4acf664d5ab05ed79686968a06,1_cc438_z003,chr11_32,11.0,rs371701829,57864175.0,0.321240,CT,C,0.053399,0.008821,6.0537,1.431858e-09,Unnamed
855,b6a7eedfc0e84b8b9d1f099ed5f158c4,1_b6a7e_z009,chr11_37,11.0,rs7119588,66377035.0,0.057960,T,C,-0.109830,0.017633,-6.2285,4.767602e-10,Unnamed
458,64bde9a562c24419b0cc96b1e922ec95,1_64bde_z000,chr12_19,12.0,rs3741760,28544464.0,0.755160,G,A,0.066336,0.009250,7.1713,7.585776e-13,CCDC91*
653,9976e04864744db58cabdef4a6c473fc,1_9976e_z001,chr12_47,12.0,rs764004172,78199435.0,0.431320,TAGTA,T,-0.049736,0.008058,-6.1722,6.813967e-10,Unnamed
346,4b7658276c0c4f12a75d95f67d7f3aa8,1_4b765_z000,chr12_67,12.0,rs35350651,111907431.0,0.505280,A,AC,0.053989,0.007948,6.7927,1.119438e-11,Unnamed
515,7382663c96c44c00a5bb6d119776c01b,1_73826_z000,chr12_69,12.0,rs4767239,114816548.0,0.825660,G,C,0.073146,0.010610,6.8942,5.520774e-12,TBX5


In [15]:
from PIL import Image

In [17]:
os.chdir(CARDIAC_COMA_REPO)

In [21]:
regions = { f"{assoc[1].region} ({assoc[1].P})": assoc[1].region for assoc in best_association_per_region.sort_values("P").iterrows() } 

@interact
def manhattan(region=widgets.Select(options=regions)):
    
    from PIL import Image
    best_association_per_region
    _best_association_per_region = best_association_per_region.set_index("region")
    
    assoc = _best_association_per_region.loc[region]
    
    run_id, z_variable = assoc.run, assoc.pheno[-4:]
    
    manhattan_file = f"mlruns/1/{run_id}/artifacts/GWAS/figures/{z_variable}__manhattan.png"
    qq_file = f"mlruns/1/{run_id}/artifacts/GWAS/figures/{z_variable}__QQ-plot.png"

    display(Image.open(manhattan_file))
    display(Image.open(qq_file))

interactive(children=(Select(description='region', options={'chr6_78 (1.79473362683253e-22)': 'chr6_78', 'chr6…

#### Filter GWAS files for selected SNPs

In [ ]:
for file, info in GWAS_dict.items():
    
    prefix = info["prefix"]
    phenotype = info["phenotype"].replace(" ", "_")
    
    o_filename = gwas_harmonized_pattern.format(prefix=prefix, phenotype=phenotype)           
    ofile_selected = gwas_selected_snps_pattern.format(prefix=prefix, phenotype=phenotype)           
    
    df = pd.read_csv(o_filename, sep="\t")
    keep = df.apply(lambda row: row.SNP in best_snps, axis=1)    
    
    print(ofile_selected)
    df[keep].to_csv(ofile_selected, sep="\t", index=False, header=True)        

In [ ]:
pp = []

for file, info in GWAS_dict.items():
    
    prefix = info["prefix"]
    phenotype = info["phenotype"].replace(" ", "_")
        
    ofile_selected_snps = gwas_selected_snps_pattern.format(prefix=prefix, phenotype=phenotype)
    
    df = pd.read_csv(ofile_selected, sep="\t")
    df = df[["SNP", "P"]]
    df = df.rename({"P": f"{prefix}__{phenotype}"}, axis=1)
    
    pp.append(df)
    

pp = [dd.set_index("SNP") for dd in pp]
snps_across_gwas = functools.reduce(lambda df1,df2: pd.merge(df1,df2,on='SNP'), pp)

# WHAT'S HAPPENING WITH THIS SNP?
snps_across_gwas = snps_across_gwas.drop("rs533885")
log10p_gwas_df = (-np.log10(snps_across_gwas))
log10p_gwas_df.to_csv(LOGP_PATH, index=True)

In [ ]:
log10p_gwas_df = pd.read_csv(LOGP_PATH)
log10p_gwas_df